In [ ]:
from itertools import chain
from pathlib import Path

from IPython.display import Markdown, display
from tqdm.notebook import tqdm

import hashlib
from os import environ
import subprocess

import githelpers
import ghapicache
import release_notes

%load_ext autoreload
%autoreload 1
%aimport githelpers
%aimport ghapicache
%aimport release_notes

ReleaseMetadata = release_notes.ReleaseMetadata

In [2]:
cached = ghapicache.GhApiCache(repo="celeritas")
ghapi = cached.api
# cached.purge()

# Load teams and cache pull request

In [17]:
# Load teams
TEAMS = {}
for t in tqdm(cached.teams()):
    TEAMS[t['name']] = release_notes.get_team(cached, t)

team_members = frozenset(chain(*(t.members for t in TEAMS.values())))
org_members = frozenset(m['login'] for m in cached.org_members())
TEAMS['affiliates'] = release_notes.Team(description="Associated with Celeritas but not core members",
                                         members=(org_members - team_members))

# Initialize the user cache
local_repo = Path("/Users/seth/Code/celeritas-temp")
user_cache = release_notes.UserCache(cached, local_repo / "scripts/release/users.json")
for login in tqdm(org_members):
    user_cache[login]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

# List active members

This is to be used for crediting in presentations, etc.

In [4]:
text = []
for team, title in [
    ('code-lead', 'Code lead'),
    ('core', 'Core members'),
    ('core-advisor', 'Core advisors'),
    ('affiliates', 'Affiliates'),]:
    text.append(f"## {title}")
    m = [user_cache[username] for username in TEAMS[team].members]
    for member in sorted(m, key=release_notes.get_last_name):
        text.append("- " + release_notes.format_user(member))
    text.append("")

display(Markdown("\n".join(text)))

## Code lead
- Seth R. Johnson *(ORNL)*

## Core members
- Elliott Biondo *(ORNL)*
- Julien Esseiva *(LBNL)*
- Hayden Hollenbeck *(UVA)*
- Seth R. Johnson *(ORNL)*
- Soon Yung Jun *(FNAL)*
- Guilherme Lima *(FNAL)*
- Amanda Lund *(ANL)*
- Ben Morgan *(U Warwick)*
- Sakib Rahman *(BNL)*
- Stefano Tognini *(ORNL)*

## Core advisors
- Philippe Canal *(FNAL)*
- Marcel Demarteau *(ORNL)*
- Tom Evans *(ORNL)*

## Affiliates
- Lance Bullerwell *(ORNL)*
- Wouter Deconinck *(U Manitoba)*
- Sam Eriksen *(U Bristol)*
- Steven Hamilton *(ORNL)*
- Kevin Pedro *(None)*
- Paul Romano *(ANL)*
- Frederic Suter *(ORNL)*
- Sandro Wenzel *(CERN)*


In [28]:
# All commits from all time
all_md = ReleaseMetadata(
    merge_bases=['v0.0.0'],
    target_branch='upstream/develop',
)

all_prs = release_notes.PullRequestRange(all_md)
count_all_contrib = release_notes.ContributionCounter(cached)
for pr in tqdm(all_prs.pull_ids):
    count_all_contrib(pr)
all_contrib_authors = count_all_contrib.sorted().author

Can't match log subject to PR: Format code base (clang-format version 11.0.1)


  0%|          | 0/1595 [00:00<?, ?it/s]

In [29]:
print("Missing ORCIDs:")
print(" ".join(f"@{u}" for u in all_contrib_authors if user_cache[u].orcid is None))

Missing ORCIDs:
@VHLM2001 @Rashika-Gupta @DoaaDeeb


In [8]:
cached.flush()

Saved cache to: data/ghapicache-celeritas-project-celeritas.json


# Release process

- Merge base should be all commits *already* released (skip documenting)
- Target branch is the one where the release candidate is
- Previous major branch allows all "v.x" contributors to be credited

In [9]:
# Major release
major_md = release_notes.ReleaseMetadata(
    release='0.6.0',
    merge_bases=['v0.5.0', 'v0.5.3'],
    target_branch='develop'
)

# Minor release
minor_md = release_notes.ReleaseMetadata(
    release='0.6.1',
    merge_bases=['v0.6.0'],
    target_branch='backports/v0.6'
)

## Release note generation

In [10]:
if 1:
    # Backport release:
    release_md = minor_md
    note_body =  """
Version {release} is a minor capability and bug fix update. Geant4-to-ORANGE
conversion has gained new abilities including:

- Support for reflection
- Support for replicated and parameterized volumes
- New shapes including G4ExtrudedSolid, G4GenericPolycone, G4Paraboloid,
  polar-truncated sphere
- Fixes and improvements to deduplication of general quadric surfaces such as
  twisted trap faces and ellipsoids

Geant4 integration has improved with:

- Automatic generation of field maps from in-memory G4Field
- Track IDs are now reconstructed when returning hits to Geant4
- Track weights are propagated through Celeritas and returned to Geant4
- Additional data members are reconstructed in hit track/steps to reduce null
  pointers
- G4VTrackUserInfo and G4VProcess are reconstructed as valid (but not
  necessarily meaningful) pointers

Code compatibility improvements include:

- Support for CUDA 12.9 and NVCC with cxxstd=20
- Changes to allow GCC's overzealous ``-Wmissing-braces`` (#1863) used by CMSSW
- Fixes to include-what-you-use errors that failed newer compilers/c++ libs
- Support for CMake ``find_package(Celeritas)`` multiple times (still
  discouraged though)
- Allow multiple runs when using GPU
- Improved compatibility for Geant4 10
- Improved use of environment variables (now the *value* of ``CELER_DISABLE``
  and others are considered, not just their existence)

"""
else:
    # Major release
    release_md = major_md
    note_body = """
Version {release} is a major update to Celeritas featuring:

-

A few minor features are noteworthy:

- 

Notable bug fixes include:

- 

Some interfaces have been removed:

- 

Known deficiencies:

- 
"""

In [11]:
prs = release_notes.PullRequestRange(release_md)
sorted_pulls = release_notes.SortedPulls(cached)
count_contrib = release_notes.ContributionCounter(cached)
for pr_id in tqdm(prs.pull_ids):
    try:
        count_contrib(pr_id)
        sorted_pulls.add(pr_id)
    except Exception as e:
        print(f"Error adding PR #{pr_id}: {e}")
        del cached.cache["pull"][cached.subkey(pr_id)]

reviewers = count_contrib.sorted().reviewer
for login in tqdm(reviewers):
    user_cache[login]

Can't match log subject to PR: Write setup Geant4 log messages to world logger, not self logger


  0%|          | 0/112 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [14]:
def make_notes(cls, release_md, note_body):
    prev = release_md.merge_bases[0]
    other = release_md.merge_bases[1:]
    if not other:
        change_str = f"Changes since {prev} follow."
    else:
        # NOTE: this is redundant if prev is a merge base of other
        assert len(other) == 1
        change_str = f"Changes since {prev}, excluding those released in {other[0]}, follow."

    notes = cls(release_md, note_body)
    notes.paragraph(change_str)
    notes.sorted_pulls(sorted_pulls)
    notes.reviewers(reviewers, user_cache)
    notes.changelog_line("celeritas-project", "celeritas")

    return notes

In [15]:
rst_notes = make_notes(release_notes.RstNotes, release_md, note_body)

with githelpers.open_pbcopy() as pb:
    rst_notes.write(pb)

print("Release notes copied to clipboard")
subprocess.run(["open", str(githelpers.REPO / "doc/release-history")])

Release notes copied to clipboard


CompletedProcess(args=['open', '/Users/seth/Code/celeritas-temp/doc/release-history'], returncode=0)

## Create github release

- First run creates draft
- After publish, re-run to get release
- Then create new spack version

In [20]:
ghapi = cached.api

markdown_notes = make_notes(release_notes.MarkdownNotes, release_md, note_body)

gh_release = release_notes.find_release(ghapi, release_md.release)
if not gh_release:
    gh_release = release_notes.create_release(ghapi, release_md, str(markdown_notes))

In [22]:
tarball = release_notes.get_or_upload_tarball(cached, gh_release)
print(f"Artifact URL: {tarball.url}")

No tarball found in release assets: reloading from GitHub
Loading https://github.com/celeritas-project/celeritas/releases/download/v0.6.1/celeritas-0.6.1.tar.gz from cached file data/ghapicache-downloads/36423d209583dcabdde7651f28c370e25aaf62c0.gz
Found tarball in release assets
Artifact URL: https://github.com/celeritas-project/celeritas/releases/download/v0.6.1/celeritas-0.6.1.tar.gz


In [23]:
# assert 0
sha256_hash = hashlib.sha256(tarball.content).hexdigest()
with githelpers.open_pbcopy() as pb:
    pb.write(f'version("{release_md.release}", sha256="{sha256_hash}")\n')
print("Spack version copied to clipboard!")
spack = Path(environ["SPACK_ROOT"]) / "bin" / "spack"
package_dir = Path(
    subprocess.check_output([spack, "location", "-p", cached.repo]).decode().strip()
)
subprocess.check_call(["open", package_dir / "package.py"])

Spack version copied to clipboard!


0

## Print authors since the last major release for presentations

In [30]:
# All commits from all time
latest_major_md = ReleaseMetadata(
    merge_bases=['v0.6.0-dev'],
    target_branch='backports/v0.6',
)

latest_prs = release_notes.PullRequestRange(latest_major_md)
count_contrib = release_notes.ContributionCounter(cached)
for pr in tqdm(latest_prs.pull_ids):
    count_contrib(pr)
authors = count_contrib.sorted().author

text = ["# Current contributors"]
current_contrib = set()
for username in authors:
    m = user_cache[username]
    text.append(f"- {m.name} (@{m.github})")
    current_contrib.add(m.github)
text.extend(["", "# Past contributors"])
for username in all_contrib_authors:
    if username not in current_contrib:
        m = user_cache[username]
        text.append(f"- {m.name} (@{username})")
display(Markdown("\n".join(text)))

Can't match log subject to PR: Write setup Geant4 log messages to world logger, not self logger


  0%|          | 0/346 [00:00<?, ?it/s]

# Current contributors
- Seth R. Johnson (@sethrj)
- Amanda Lund (@amandalund)
- Julien Esseiva (@esseivaju)
- Elliott Biondo (@elliottbiondo)
- Philippe Canal (@pcanal)
- Hayden Hollenbeck (@hhollenb)
- Stefano Tognini (@stognini)
- Lance Bullerwell (@lebuller)
- Soon Yung Jun (@whokion)
- Guilherme Lima (@mrguilima)
- Rashika Gupta (@Rashika-Gupta)
- Damien L-G (@dalg24)
- Sakib Rahman (@rahmans1)
- Ben Morgan (@drbenmorgan)

# Past contributors
- Paul Romano (@paulromano)
- Vincent R. Pascuzzi (@vrpascuzzi)
- Tom Evans (@tmdelellis)
- Vidor Heli Lujan Montiel (@VHLM2001)
- Doaa Deeb (@DoaaDeeb)
- Andrey Prokopenko (@aprokop)
- Shane Hart (@hartsw)
- Peter Heywood (@ptheywood)

## Push to zenodo

In [24]:
import zenodoapi
%aimport zenodoapi

In [25]:
# Load the Zenodo token
token_path = Path.home() / ".config/zenodo-token"
with open(token_path) as f:
    zenodo_token = f.read().strip()

zenodo = zenodoapi.Zenodo(zenodo_token)
zenodo.api_url = "https://zenodo.org/api/"

In [26]:
def load_contributions(release_md):
    prs = release_notes.PullRequestRange(release_md)
    count_contrib = release_notes.ContributionCounter(cached)
    for pr in tqdm(prs.pull_ids):
        count_contrib(pr)

    # Create author list
    return count_contrib.sorted()

def download_tarball(gh_release):
    asset = gh_release['assets'][0]
    name = asset['name']
    content = cached.download_file(asset['url'], ext=Path(name).suffix)
    assert content is not None
    return (content, name)

make_zenodo_md = release_notes.ZenodoMetadataBuilder(user_cache=user_cache, teams=TEAMS)

### Minor release

This assumes the major release has already been published upstream into Zenodo.

In [27]:
assert not release_md.is_major()
major, minor, patch = release_md.as_version()
release_md = release_notes.ReleaseMetadata.from_comprehensive_version(minor, patch)
major_dep = zenodo.find_deposition(f"Celeritas {major}.{minor}")

In [28]:
zenodo.get_deposition(major_dep.get_latest_version().id)

ZenodoDeposition(id=15281110, title="Celeritas 0.6")

In [29]:
gh_release = release_notes.find_release(ghapi, release_md.release)

In [31]:
# Get the Zenodo metadata    
from requests import HTTPError


old_md = major_dep.data["metadata"]
new_md = make_zenodo_md(load_contributions(release_md), release_md, gh_release)
# Only update editors, not team members
old_contrib = new_md["contributors"]
new_contrib = [u for u in old_contrib if u["type"] == "Editor"]
old_contrib = [u for u in old_contrib if u["type"] != "Editor"]
new_md["contributors"] = new_contrib + old_contrib
# Don't change the title
new_md["title"] = major_dep.data["metadata"]["title"]
# Add the body
new_md["description"] = "\n\n".join(
    [
        old_md["description"],
        f"<h1>Version {release_md.release}</h1>",
        new_md["description"],
    ]
)
try:
    new_vers = major_dep.create_new_version()
except HTTPError as e:
    # Maybe the version already exists
    if e.response.status_code == 400:
        print("Draft may already exist:", e)
        new_vers = major_dep.get_latest_draft()
    else:
        raise
try:
    new_vers.update(new_md)
except HTTPError as e:
    # Something else went wrong? Try to upload the tarball
        print("Draft may already exist:", e)

# Upload the release
(content, name) = download_tarball(gh_release)
new_vers.upload(content, name)
# The old tarball may still be there (this is buggy) so delete it
new_vers.refresh()
for file in new_vers.get_files():
    if file.filename != name:
        try:
            file.delete()
        except Exception as e:
            print(f"Failed to delete {file.filename}: {e}")
new_vers.refresh()

Can't match log subject to PR: Write setup Geant4 log messages to world logger, not self logger


  0%|          | 0/347 [00:00<?, ?it/s]

Created new version 17114294: https://zenodo.org/deposit/17114294
Updated deposition at https://zenodo.org/deposit/17114294 : Celeritas 0.6
Uploaded celeritas-0.6.1.tar.gz: version 01ffac04-7d92-4303-9b16-42fd041f0fe4
Failed to delete celeritas-0.6.0.tar.gz: Expecting value: line 1 column 1 (char 0)


ZenodoDeposition(id=17114294, title="Celeritas 0.6")

## Major release

In [57]:
assert release_md.is_major()
major, minor, patch = release_md.as_version()
release_md = release_notes.ReleaseMetadata.from_comprehensive_version(minor)
title = f"Celeritas {major}.{minor}"

In [58]:
zmd = make_zenodo_md(load_contributions(release_md), release_md, gh_release)
zmd["title"] = title

# Create the Zenodo deposition
deposition = zenodo.create_deposition(zmd)

  0%|          | 0/234 [00:00<?, ?it/s]

Created deposition 15281110 at https://zenodo.org/deposit/15281110 : Celeritas 0.6


In [60]:
# Upload the release tarball
deposition.upload(artifact_tgz, f"celeritas-{release_md.release}.tar.gz")

print(deposition.html)
print("""
IMPORTANT Checklist:
- Add community (CANNOT be done later)
- Update contributors based on release date
- Update licenses
""")

Uploaded celeritas-0.6.0.tar.gz: version bbd63016-30a1-4be9-9a74-329ca975bed9
https://zenodo.org/deposit/15281110

IMPORTANT Checklist:
- Add community (CANNOT be done later)
- Update contributors based on release date
- Update licenses

